# 从0开始的RNN-文本预处理
例如，一篇文章可以被简单地看作是一串单词序列，甚至是一串字符序列。 本节中，我们将解析文本的常见预处理步骤。 这些步骤通常包括：
1. 将文本作为字符串加载到内存中。
2. 将字符串拆分为词元（如单词和字符）。
3. 建立一个词表，将拆分的词元映射到数字索引。
4. 将文本转换为数字索引序列，方便模型操作。

## 读取文本行
url: https://www.gutenberg.org/ebooks/35  
离线文件相对地址: ../data/TimeMachine.txt
涉及知识点-python正则匹配  
re.sub(pattern, rep, inp) 是一种带正则的字符串替换函数，功能是将输入字符串inp中所有与pattern模式匹配的子串替换为rep，例如`re.sub("\d+", "6", inp)`就是把字符串中所有的数字都换为6.  
[^a-z]可以匹配任何不在“a”到“z”范围内的任意字符。
参考链接:  
[python re模块(正则表达式) sub()函数详解](https://blog.csdn.net/qq_43088815/article/details/90214217)  
[正则表达式符号大全](https://blog.csdn.net/wujunlei1595848/article/details/81316800)

In [3]:
# 读取文件, 转为一个字符串行列表, 去掉除字母空格以外的字符并统一转为小写
import re
def read_text(url=None):
    if not url:
        url = "../data/TheTimeMachine.txt"
    with open(url, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    return [re.sub(r"[^a-zA-Z]+", " ", line).strip().lower() for line in lines]
lines = read_text()
len(lines), lines[0], lines[1], lines[2]

(3557,
 'the project gutenberg ebook of the time machine by h g wells',
 '',
 'this ebook is for the use of anyone anywhere in the united states and')

## 词元(token)列表
以上，将文本的每一行作为一个字符串读入了内存，并将所有非字母字符替换为空格，最后去掉首尾多余的空格，并将所有字母转为小写。以下是把列表中的每一个元素--一行（看做一个字符串）转为一个**词元(token)列表**，它可以是一个单词或者字符，默认是单词。下面的tokenize函数将文本行列表（lines）作为输入， 列表中的每个元素是一个文本序列（如一条文本行）。 每个文本序列又被拆分成一个词元列表，词元（token）是文本的基本单位。 最后，返回一个由词元列表组成的列表，其中的每个词元都是一个字符串（string）

In [5]:
def tokenize(lines, token="word"):
    if token == "word":
        return [line.split() for line in lines]
    if token == "char":
        return [list(line) for line in lines]
tokens = tokenize(lines)
print(tokens[0])

['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'time', 'machine', 'by', 'h', 'g', 'wells']


## 词表建立
词元的类型是字符串，而模型需要的输入是数字，因此这种类型不方便模型使用。 现在，让我们构建一个字典，通常也叫做词表（vocabulary）， 用来将字符串类型的词元映射到从0开始的数字索引中。 我们先将训练集中的所有文档合并在一起，对它们的唯一词元进行统计， 得到的统计结果称之为语料（corpus）。 然后根据每个唯一词元的出现频率，为其分配一个数字索引。 很少出现的词元通常被移除，这可以降低复杂性。 另外，语料库中不存在或已删除的任何词元都将映射到一个特定的未知词元“< unk >”。 我们可以选择增加一个列表，用于保存那些被保留的词元， 例如：填充词元（“< pad >”）； 序列开始词元（“< bos >”）； 序列结束词元（“< eos >”）    

[Python collections.Counter()用法](https://blog.csdn.net/qwe1257/article/details/83272340)  
collections在python官方文档中的解释是High-performance container datatypes，直接的中文翻译解释高性能容量数据类型。其中Counter中文意思是计数器，也就是我们常用于统计的一种数据类型，在使用Counter之后可以让我们的代码更加简单易读。  

[Python @property装饰器详解](http://c.biancheng.net/view/4561.html)  
既要保护类的封装特性，又要让开发者可以使用“对象.属性”的方式操作操作类属性，Python提供了 @property 装饰器。通过 @property 装饰器，可以直接通过方法名来得到属性值（方法返回值），不需要在方法名后添加一对“（）”小括号。 

[Python 字典(Dictionary) get()方法](https://www.runoob.com/python/att-dictionary-get.html)  
Python 字典(Dictionary) get() 函数返回指定键的值。
dict.get(key[, value])   
key -- 字典中要查找的键。  
value -- 可选，如果指定键的值不存在时，返回该默认值。  

In [18]:
import collections
class Vocab:
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 将tokens展平为一个单词列表，对其中的单词进行计数并按词频倒序排序
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), 
                                   key=lambda x: x[1], reverse=True)
        # id->token, token->id
        # 先处理保留词元，例如未定义词语
        self.id2token = ['<unk>']+reserved_tokens
        self.token2id = {token: idx for idx, token in enumerate(self.id2token)}
        # 接下来对_token_freqs进行处理
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            self.id2token.append(token)
            self.token2id[token]=len(self.id2token)-1
    
    def __len__(self):
        return len(self.id2token)
    # 通过token获取id, 注意这是一个递归操作, 也即是可以传入列表的
    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            # dict().get(key, default_value)
            return self.token2id.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.id2token[indices]
        return [self.to_tokens[index] for index in indices]

    @property
    def unk(self):  # 未知词元的索引为0
        return 0
    # @property使得外部类可以通过函数名访问私有属性
    @property
    def token_freqs(self):
        return self._token_freqs    

def count_corpus(tokens):
    """词频计数"""
    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

dict.items()
https://www.runoob.com/python/att-dictionary-items.html  
以列表返回可遍历的(键, 值) 元组数组

In [19]:
vocab = Vocab(tokens)
list(vocab.token2id.items())[:10]

[('<unk>', 0),
 ('the', 1),
 ('and', 2),
 ('of', 3),
 ('i', 4),
 ('a', 5),
 ('to', 6),
 ('in', 7),
 ('was', 8),
 ('that', 9)]

In [20]:
list(vocab.token_freqs)[:10]

[('the', 2477),
 ('and', 1312),
 ('of', 1286),
 ('i', 1268),
 ('a', 877),
 ('to', 766),
 ('in', 606),
 ('was', 554),
 ('that', 458),
 ('it', 452)]

In [25]:
for i in range(3):
    print(tokens[i])
    print(vocab[tokens[i]])
vocab.token_freqs[vocab["flower"]-1]

['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[1, 53, 44, 314, 3, 1, 19, 46, 33, 1163, 1164, 360]
[]
[]
['this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'united', 'states', 'and']
[21, 314, 29, 17, 1, 220, 3, 558, 1165, 7, 1, 268, 235, 2]


('flower', 1)

In [26]:
def load_corpus_time_machine(max_tokens=-1):
    lines = read_text()
    tokens = tokenize(lines, token="char")
    vocab = Vocab(tokens)
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    corups = [vocab[token] for line in tokens for token in line]
    return corups, vocab
corups, vocab = load_corpus_time_machine()

In [27]:
len(corups), len(vocab)

(189663, 28)

In [34]:
vocab.id2token[:10]

['<unk>', ' ', 'e', 't', 'a', 'i', 'o', 'n', 's', 'r']

In [35]:
list(vocab.token_freqs)[:5]

[(' ', 32926), ('e', 19781), ('t', 15155), ('a', 12752), ('i', 11312)]

In [36]:
l0 = corups[0]
t0 = vocab.id2token[l0]
t0a = vocab.to_tokens(l0)
l0a = vocab[t0]
l0, t0, t0a, l0a

(3, 't', 't', 3)